In [1]:
import os, sys
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:   
    sys.exit("please declare environment variable 'SUMO_HOME'")
    
import sumolib

In [2]:
import pickle

with open('data/small_features.pkl', 'rb') as f:
    features = pickle.load(f)

with open('data/small_speeds.pkl', 'rb') as f:
    speeds = pickle.load(f)
    
with open('data/small_coords.pkl', 'rb') as f:
    coords = pickle.load(f) 

In [3]:
import pandas as pd

edges = pd.read_csv("data/small.csv", names=["source", "dest"])
edges.head()

,source,dest
0,1614922744,1614922747
1,269459339,269459341
2,1621320618,1621320622
3,269459229,269459230
4,607954833,1613371666


In [4]:
# Create node XML
import xml.etree.cElementTree as ET

nodes = ET.Element("nodes")
for n in set(edges["source"]).union(set(edges["dest"])):
    v = str(n)
    long, lat = coords[v]
    node = ET.SubElement(nodes, "node", id=v, x=str(long), y=str(lat))
    
tree = ET.ElementTree(nodes)
tree.write("out/small.nodes.xml")

In [5]:
# Create edge XML

edgeXML = ET.Element("edges")
i = 0
for i, e in enumerate(edges.itertuples()):
    feat = features[tuple([str(v) for v in e[1:3]])]
    numLanes = str(int(feat[2]))
    ET.SubElement(edgeXML, "edge", {"id": str(i), "from": str(e[1]), "to": str(e[2]), 
                                    "numLanes": str(int(feat[2])), "speed": str(feat[3] * 0.44704), "length": str(feat[4])})
    
tree = ET.ElementTree(edgeXML)
tree.write("out/small.edges.xml")

In [17]:
# Run netconvert first

import numpy as np
from tqdm.notebook import tqdm

net = sumolib.net.readNet('out/small.net.xml')

with open('data/small_flows.pkl', 'rb') as f:
    flows = pickle.load(f)

readings = pd.DataFrame(
    {
        "Detector": [],
        "Time": [],
        "qPKW": [],
        "vPKW": []
    }
)

detectors = ET.Element("detectors")
detector_keys = set(speeds.keys()).intersection(flows.keys())

for i, e in tqdm(enumerate(net.getEdges())):
    key = (str(e.getFromNode().getID()), str(e.getToNode().getID()))

    if key in detector_keys:
        # Add detectors to XML
        for lane in e.getLanes():
            ET.SubElement(detectors, "detectorDefinition", id="det" + str(lane.getID()), 
                          lane=str(lane.getID()), pos=str(lane.getLength() / 2))
        
            # Add info to csv
            avg_speed = speeds[key] * 0.44704 # Convert from mph to m/s
            avg_flow = flows[key] / (len(e.getLanes()))
            for j in range(1, 2):
#                 speed, flow = -1, -1
#                 while speed < 0:
#                     speed = avg_speed * np.random.default_rng().normal(1, 0.25)
#                 while flow < 0:
#                     flow = avg_flow * np.random.default_rng().normal(1, 0.25)
                
                readings.loc[len(readings.index)] = ["det" + str(lane.getID()), 60 * j, int(avg_flow), avg_speed] 
        
tree = ET.ElementTree(detectors)
tree.write("out/small.detectors.xml")
readings.to_csv("out/small.readings.csv", sep=";", index=False)

readings.head()

0it [00:00, ?it/s]

,Detector,Time,qPKW,vPKW
0,det1046_0,60.0,833.0,29.307504
1,det1046_1,60.0,833.0,29.307504
2,det1046_2,60.0,833.0,29.307504
3,det1046_3,60.0,833.0,29.307504
4,det1046_4,60.0,833.0,29.307504


In [7]:
readings[readings["Detector"] == "det1046_0"]["qPKW"].mean()

833.0

In [18]:
import xml.etree.cElementTree as ET
import numpy as np

net = sumolib.net.readNet('out/small.net.xml')

with open('data/small_flows.pkl', 'rb') as f:
    flows = pickle.load(f)
    
data = ET.Element("data")

for t in range(3):
    interval = ET.SubElement(data, "interval", id=str(t), begin=str(3600 * t), end=str(3600 * (t + 1)))
    
    for i, e in enumerate(net.getEdges()):
        key = (str(e.getFromNode().getID()), str(e.getToNode().getID()))
        if key in flows.keys():
            flow = str(int(flows[key]))
            speed = speeds[key] * 0.44704 # Convert from mph to m/s
            ET.SubElement(interval, "edge", id=str(e.getID()), entered=flow, speed=speed)
            
            
tree = ET.ElementTree(data)
tree.write("out/routeSampler/small.edgeData.xml")

KeyError: ('1614922929', '268539761')

In [15]:
out_detectors = ET.Element("additional")

for e in net.getEdges():
    lane=np.random.choice(e.getLanes())
    det = ET.SubElement(out_detectors, "inductionLoop", id="out" + str(e.getID()), 
                        lane=str(lane.getID()),
                        pos=str(lane.getLength() / 2),
                        period=str(60 * 60 * 24),
                        file="small.detectors.out.xml")

tree = ET.ElementTree(out_detectors)
tree.write("out/small.detectors.additional.xml")